<a href="https://colab.research.google.com/github/sanazgit/Machin-Learning/blob/main/ELM_ProblemFourClasses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Extreme Learning Machine for Four_Classes problem**

---



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pylab import *
import numpy as np
import pandas as pd
from numpy import linalg as LA
import math
from sklearn.preprocessing import LabelEncoder
import statistics


# **ELM tanh class**




In [ ]:
class ELM_tanh(object):  
    
    def __init__(self, inputSize, outputSize, hiddenSize):

        self.inputSize = inputSize
        self.outputSize = outputSize
        self.hiddenSize = hiddenSize       
        
        # Initialize random weight with range [-0.1, 0.5]
        self.weight = np.matrix(np.random.uniform(-0.2, 0.5, (self.hiddenSize, self.inputSize)))

        # Initialize random bias with range [0, 0.5]
        self.bias = np.matrix(np.random.uniform(0, 1, (1, self.hiddenSize)))
        
        self.H = 0
        self.beta = 0

    def tanh(self, x):
        """
         tanh or Hyperbolic activation function
        """
        return (np.exp(x)-np.exp(-x))/(np.exp(x)+np.exp(-x))

    def predict(self, X):
        """
        Predict the results of the training process using test data
        """
        X_predicted=[]
        X = np.matrix(X)
        y = np.dot(self.tanh((X * self.weight.T) + self.bias) , self.beta)
        y=np.array(y)

        for ym in y:
          X_predicted.append(ym.argmax())

        return X_predicted

    def train(self, X, y):
        """
        Extreme Learning Machine training process  
        """
        X = np.matrix(X)

        #y = np.matrix(y)
        one_hot_labels = np.zeros((X.shape[0], 4))

        for i in range(X.shape[0]):
          one_hot_labels[i, y[i]] = 1        
        
        # Calculate hidden layer output matrix (Hinit)
        self.H = (X * self.weight.T) + self.bias

        # tanh activation function
        self.H = self.tanh(self.H)

        # Calculate the Moore-Penrose pseudoinverse matriks        
        H_moore_penrose = np.linalg.pinv(self.H) 

        # Calculate the output weight matrix beta
        self.beta = np.dot(H_moore_penrose, one_hot_labels)

        output= self.H * self.beta

* **Read Data**

In [ ]:
# Read train data
data= pd.read_excel('/content/drive/MyDrive/Colab Notebooks/ML_Exercises/MultiClass_problem_train.xlsx')
data

,x1,x2,y
0,0.837747,0.676387,1
1,1.295292,0.793018,1
2,0.810387,0.858808,1
3,0.722597,0.942760,1
4,0.788612,1.213929,1
...,...,...,...
3995,1.930744,1.739438,4
3996,1.648980,2.044412,4
3997,1.862512,1.549823,4
3998,1.933452,1.966226,4


In [ ]:
y_train = data['y'].values  #segregating the label vlue from the feature value.
X_train = data.drop(['y'],axis=1).values
X_train

array([[0.83774656, 0.67638741],
       [1.29529151, 0.79301754],
       [0.81038668, 0.85880804],
       ...,
       [1.86251242, 1.5498227 ],
       [1.93345221, 1.96622559],
       [1.61655592, 1.8752228 ]])

In [ ]:
# Read test data
data_test= pd.read_excel('/content/drive/MyDrive/Colab Notebooks/ML_Exercises/MultiClass_Problem_Test.xlsx')

y_test = data_test['y'].values  #segregating the label vlue from the feature value.
X_test = data_test.drop(['y'],axis=1).values

* **Make instance of ELM tanh class**

In [ ]:
elm = ELM_tanh(X_train.shape[1], 4,200 )

# Train data
labelencoder = LabelEncoder()
ytrain= labelencoder.fit_transform(y_train)
elm.train(X_train,ytrain)

* **Try the model with test data**

In [ ]:
ytest= labelencoder.fit_transform(y_test)
y_pred= elm.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
print('Accuracy: ', accuracy_score(ytest, y_pred))

Accuracy:  0.95


# **ELM with Gaussian_kernel**

In [ ]:
class ELM_Gaussian_kernel(object):  
    
    def __init__(self, inputSize, outputSize, hiddenSize):
      self.inputSize = inputSize
      self.outputSize = outputSize
      self.hiddenSize = hiddenSize     
      self.H = 0
      self.beta = 0

    def compute_RBF(self,mat1,mat2,sigma):
      
      """
      RBF activation function
      """
      mat1=np.mat(mat1)
      mat2= np.mat(mat2)

      trnorms1 = np.mat([(v * v.T)[0,0] for v in mat1]).T
      trnorms2 = np.mat([(v * v.T)[0,0] for v in mat2]).T

      k1 = trnorms1 * np.mat(np.ones((mat2.shape[0], 1), dtype=np.float64)).T

      k2 = np.mat(np.ones((mat1.shape[0], 1), dtype=np.float64)) * trnorms2.T

      k = k1 + k2

      k = k - 2 * (np.dot(mat1,mat2.T))

      k = k * - 1./(2 * np.power(sigma, 2))

      rbf= np.exp(k)
  
      return rbf

     
    def predict(self, X):
      """
      Predict the results of the training process using test data
      """
      dmean=[]
      X_predicted=[]

      dmax = [[ 0 for x in range(self.hiddenSize)] for y in range(self.hiddenSize)]
      for i in range(X.shape[0]):
        a=min(X[i])
        b= max(X[i])
        mue= a + (b - a) * rand(self.hiddenSize,X.shape[1]) # make a Goussian with mue center

      for i in range(len(mue)):
        for j in range(len(mue)):
          if i!=j :
            dmax[i][j]= (LA.norm(mue[i]- mue[j]))
        dmean.append(statistics.mean(dmax[i]))

      dmean= np.array(dmean)
      sigma= dmean / math.sqrt(2 * self.hiddenSize)  

      H= self.compute_RBF(X,mue,sigma)

      y_pred= np.dot(H , self.beta)
      y=np.array(y_pred)
      
      for ym in y:
        X_predicted.append(ym.argmax())

      return X_predicted

    def train(self, X, y):
      """
      Extreme Learning Machine training process  
      """

      one_hot_labels = np.zeros((X.shape[0], 4))

      for i in range(X.shape[0]):
        one_hot_labels[i, y[i]] = 1   

      dmean=[]
      dmax = [[ 0 for x in range(self.hiddenSize)] for y in range(self.hiddenSize)]
      for i in range(X.shape[0]):
        a=min(X[i])
        b= max(X[i])
        mue= a + (b - a) * randn(self.hiddenSize,X.shape[1]) # make a Goussian with mue center

      for i in range(len(mue)):
        for j in range(len(mue)):
          if i!=j :
            dmax[i][j]= (LA.norm(mue[i]- mue[j]))
        dmean.append(statistics.mean(dmax[i]))

      dmean= np.array(dmean)
      sigma= dmean / math.sqrt(2 * self.hiddenSize)   
      
    
      # Calculate hidden layer output matrix (Hinit)
      self.H= self.compute_RBF(X,mue,sigma)
  
      # Calculate the output weight matrix beta
      H_moore_penrose = np.linalg.pinv(self.H) 
      self.beta = np.dot(H_moore_penrose, one_hot_labels)
        
      output= np.dot(self.H , self.beta)


In [ ]:
elmG = ELM_Gaussian_kernel(X_train.shape[1], 4,200 )
elmG.train(X_train,ytrain)

 * **Try the model with test data**

In [ ]:
y_predG= elmG.predict(X_test)

In [ ]:
print('Accuracy: ', accuracy_score(ytest, y_predG))

Accuracy:  0.31925
